In [1]:
import json
import os
from functools import lru_cache, partial, reduce

import numpy as np
import pandas as pd
import sqlite_utils
from rdflib.graph import Graph
from rdflib_hdt import HDTStore
from timer import timer
from tqdm import tqdm

os.environ["DP_SKIP_NLTK_DOWNLOAD"] = "True"

import inspect
import json
import logging
import os
import re
import sys
from collections import defaultdict
from functools import reduce
from itertools import permutations, product

import numpy as np
import pandas as pd
from deeppavlov import build_model, configs
from deeppavlov.core.commands.infer import *
from deeppavlov.core.commands.utils import *
from deeppavlov.core.common.file import *
from deeppavlov.models.kbqa.wiki_parser import *
from rapidfuzz import fuzz
from scipy.special import softmax


logging.disable(sys.maxsize)

import csv
import gzip
import inspect
import logging
import math
import os
import re
import shutil
import sys
from collections import Counter, defaultdict, namedtuple
from copy import deepcopy
from datetime import datetime
from functools import partial, reduce

import editdistance
import networkx as nx
import numpy as np
import pandas as pd
import sqlite_utils
import synonyms
import torch.nn as nn
from deeppavlov import build_model, configs
from deeppavlov.core.commands.infer import *
from deeppavlov.core.common.file import *
from deeppavlov.models.kbqa.query_generator import *
from deeppavlov.models.kbqa.query_generator_base import *
from deeppavlov.models.kbqa.wiki_parser import *
from pandas.io.common import _stringify_path
from scipy.special import softmax
from sentence_transformers import InputExample, LoggingHandler, util
from sentence_transformers.util import pytorch_cos_sim
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import \
    CECorrelationEvaluator
from torch.utils.data import DataLoader
from tqdm import tqdm

from transformers import (
    AdapterConfig,
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    HfArgumentParser,
    MultiLingAdapterArguments,
    PreTrainedTokenizerFast,
    Trainer,
    TrainingArguments,
    set_seed,
)

from dataclasses import dataclass, field

import jieba
from hashlib import sha512

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    task_name: Optional[str] = field(default="ner", metadata={"help": "The name of the task (ner, pos...)."})
    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a csv or JSON file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate on (a csv or JSON file)."},
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to predict on (a csv or JSON file)."},
    )
    text_column_name: Optional[str] = field(
        default=None, metadata={"help": "The column name of text to input in the file (a csv or JSON file)."}
    )
    label_column_name: Optional[str] = field(
        default=None, metadata={"help": "The column name of label to input in the file (a csv or JSON file)."}
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to model maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    label_all_tokens: bool = field(
        default=False,
        metadata={
            "help": "Whether to put the label for one word on all tokens of generated by that word or just on the "
            "one (in which case the other tokens will have a padding index)."
        },
    )
    return_entity_level_metrics: bool = field(
        default=False,
        metadata={"help": "Whether to return all the entity levels during evaluation or just the overall ones."},
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        self.task_name = self.task_name.lower()

from ner_model import *
from tmp_classifier import *
from ranker import *

sim_model = SentenceTransformer('LaBSE')
sim_model.pool = None

assert os.path.exists("kbqa-explore/multi_lang_kb_dict.db")
wiki_entity_db = sqlite_utils.Database("kbqa-explore/multi_lang_kb_dict.db")
assert "en_zh_so_search" in wiki_entity_db.table_names()

hdt_path = "/Users/svjack/.deeppavlov/downloads/wikidata/wikidata.hdt"
wiki_parser = WikiParser(
    wiki_filename = hdt_path,
    lang = "",
)

sys.path.insert(0, "kbqa-explore")

zh_linker_entities = load_pickle("kbqa-explore/linker_entities.pkl")
zh_linker_entities.num_entities_to_return = 5

config = parse_config(configs.kbqa.kbqa_cq)

#### query template info
sparql_queries_path = pd.DataFrame(config["chainer"]["pipe"])["sparql_queries_filename"].dropna().iloc[0]
sparql_queries_df = pd.read_json(sparql_queries_path).T


def search_entity_rep_by_lang_filter_in_db(entityid, wiki_entity_db, lang = "en"):
    id = entityid
    g = wiki_entity_db.query("select * from en_zh_so_search where s = '{}' and lang = '{}'".format(id, lang))
    l = list(g)
    if not l:
        return []
    df = pd.DataFrame(l)
    l = df["o"].drop_duplicates().tolist()
    return l

class Zh_Rel_Ranker(object):
    def __init__(self, rfr_cls, pid_text_b_dict):
        assert hasattr(rfr_cls, "produce_rank_df")
        assert hasattr(rfr_cls, "sim_model")
        rfr_cls.sim_model = SentenceTransformer("LaBSE")
        assert type(pid_text_b_dict) == type({})
        self.rfr_cls = rfr_cls
        self.pid_text_b_dict = pid_text_b_dict
        self.text_b_pid_dict = dict(map(lambda t2: (t2[1], t2[0])
                                   ,self.pid_text_b_dict.items()))

    def rank_rels(self, question, ex_rels, reverse_to_id = True):
        assert type(question) == type("")
        assert type(ex_rels) == type([])
        ex_rels = list(filter(lambda x: x in self.pid_text_b_dict.keys(), ex_rels))
        ex_rels_text_list = []
        for x in ex_rels:
            text_b = self.pid_text_b_dict[x]
            ex_rels_text_list.append(text_b)
        prob_cate_df = self.rfr_cls.produce_rank_df(question, ex_rels_text_list)
        req = prob_cate_df[["cate", "prob"]].values.tolist()
        if reverse_to_id:
            req = list(map(
                lambda t2: (self.text_b_pid_dict[t2[0]] ,t2[1])
                , req))
        return req

br_cls = RFR(b_clf,
    all_cate_list=list(pid_zh_b_dict.values()),
   sim_model=sim_model
   )
b_rel_ranker = Zh_Rel_Ranker(br_cls, pid_zh_b_dict)

def query_parser_bu(question ,query_info,
                     entities_and_types_select,
                   entity_ids,
                     type_ids,
                    entities_to_leave = 10,
                     max_comb_num = 10000,
                     return_all_possible_answers = False,
                     rel_ranker = b_rel_ranker
                   ):
    question_tokens = jieba.lcut(question)
    query = query_info["query_template"].lower()

    rels_for_search = query_info["rank_rels"]
    rel_types = query_info["rel_types"]
    query_seq_num = query_info["query_sequence"]
    return_if_found = query_info["return_if_found"]
    define_sorting_order = query_info["define_sorting_order"]
    property_types = query_info["property_types"]

    query_triplets = re.findall("{[ ]?(.*?)[ ]?}", query)[0].split(' . ')
    query_triplets = [triplet.split(' ')[:3] for triplet in query_triplets]
    query_sequence_dict = {num: triplet for num, triplet in zip(query_seq_num, query_triplets)}
    query_sequence = []
    for i in range(1, max(query_seq_num) + 1):
        query_sequence.append(query_sequence_dict[i])
    triplet_info_list = [("forw" if triplet[2].startswith('?') else "backw", search_source, rel_type)
                             for search_source, triplet, rel_type in zip(rels_for_search, query_triplets, rel_types) if
                             search_source != "do_not_rank"]
    entity_ids = [entity[:entities_to_leave] for entity in entity_ids]

    rels = [find_top_rels_bu(question, entity_ids, triplet_info, wiki_parser, rel_ranker)
                    for triplet_info in triplet_info_list]
    log.debug(f"(query_parser)rels: {rels}")
    rels_from_query = [triplet[1] for triplet in query_triplets if triplet[1].startswith('?')]
    answer_ent = re.findall("select [\(]?([\S]+) ", query)
    order_info_nt = namedtuple("order_info", ["variable", "sorting_order"])
    order_variable = re.findall("order by (asc|desc)\((.*)\)", query)
    if order_variable:
        if define_sorting_order:
            answers_sorting_order = order_of_answers_sorting(question)
        else:
            answers_sorting_order = order_variable[0][0]
        order_info = order_info_nt(order_variable[0][1], answers_sorting_order)
    else:
        order_info = order_info_nt(None, None)
    print(f"question, order_info: {question}, {order_info}")
    filter_from_query = re.findall("contains\((\?\w), (.+?)\)", query)
    print(f"(query_parser)filter_from_query: {filter_from_query}")
    year = extract_year(question_tokens, question)
    number = extract_number(question_tokens, question)
    print(f"year {year}, number {number}")
    if year:
        filter_info = [(elem[0], elem[1].replace("n", year)) for elem in filter_from_query]
    elif number:
        filter_info = [(elem[0], elem[1].replace("n", number)) for elem in filter_from_query]
    else:
        filter_info = [elem for elem in filter_from_query if elem[1] != "n"]
    for unk_prop, prop_type in property_types.items():
        filter_info.append((unk_prop, prop_type))
    print(f"(query_parser)filter_from_query: {filter_from_query}")
    rel_combs = make_combs(rels, permut=False)
    import datetime
    start_time = datetime.datetime.now()
    entity_positions, type_positions = [elem.split('_') for elem in entities_and_types_select.split(' ')]
    print(f"entity_positions {entity_positions}, type_positions {type_positions}")
    selected_entity_ids = [entity_ids[int(pos) - 1] for pos in entity_positions if int(pos) > 0]
    selected_type_ids = [type_ids[int(pos) - 1] for pos in type_positions if int(pos) > 0]
    entity_combs = make_combs(selected_entity_ids, permut=True)
    type_combs = make_combs(selected_type_ids, permut=False)
    print(f"(query_parser)entity_combs: {entity_combs[:3]}, type_combs: {type_combs[:3]},"
              f" rel_combs: {rel_combs[:3]}")
    queries_list = []
    parser_info_list = []
    confidences_list = []
    all_combs_list = list(itertools.product(entity_combs, type_combs, rel_combs))

    for comb_num, combs in enumerate(all_combs_list):
        confidence = np.prod([score for rel, score in combs[2][:-1]])
        confidences_list.append(confidence)
        query_hdt_seq = [
            fill_query(query_hdt_elem, combs[0], combs[1], combs[2]) for query_hdt_elem in query_sequence]
        if comb_num == 0:
            print(f"\n__________________________\nfilled query: {query_hdt_seq}\n__________________________\n")
        queries_list.append((rels_from_query + answer_ent, query_hdt_seq, filter_info, order_info, return_if_found))
        parser_info_list.append("query_execute")
        ##if comb_num == self.max_comb_num:
        if comb_num == max_comb_num:
            break
    candidate_outputs = []
    #candidate_outputs_list = self.wiki_parser(parser_info_list, queries_list)
    candidate_outputs_list = wiki_parser(parser_info_list, queries_list)

    if isinstance(candidate_outputs_list, list) and candidate_outputs_list:
        outputs_len = len(candidate_outputs_list)
        all_combs_list = all_combs_list[:outputs_len]
        confidences_list = confidences_list[:outputs_len]
        for combs, confidence, candidate_output in zip(all_combs_list, confidences_list, candidate_outputs_list):
            candidate_outputs += [[combs[0]] + [rel for rel, score in combs[2][:-1]] + output + [confidence]
                                  for output in candidate_output]
        #if self.return_all_possible_answers:
        if return_all_possible_answers:
            candidate_outputs_dict = defaultdict(list)
            for candidate_output in candidate_outputs:
                candidate_outputs_dict[(tuple(candidate_output[0]),
                                        tuple(candidate_output[1:-2]))].append(candidate_output[-2:])
            candidate_outputs = []
            for (candidate_entity_comb, candidate_rel_comb), candidate_output in candidate_outputs_dict.items():
                candidate_outputs.append(list(candidate_rel_comb) +
                                         [tuple([ans for ans, conf in candidate_output]), candidate_output[0][1]])
        else:
            candidate_outputs = [output[1:] for output in candidate_outputs]
    print(f"(query_parser)loop time: {datetime.datetime.now() - start_time}")
    print(f"(query_parser)final outputs: {candidate_outputs[:3]}")
    return candidate_outputs

def find_top_rels_bu(question: str, entity_ids: List[List[str]],
                     triplet_info: Tuple, wiki_parser, rel_ranker,
                     rels_to_leave = 10
                     , entities_to_leave = 5, source = "wiki",
                    ):
    assert source == "wiki"
    ex_rels = []
    direction, source, rel_type = triplet_info
    if source == "wiki":
        print(triplet_info)
        print("entity_ids :")
        print(entity_ids)
        print("-" * 100)

        queries_list = list({(entity, direction, rel_type) for entity_id in entity_ids
                             for entity in entity_id[:entities_to_leave]})
        print("queries_list: {}".format(queries_list))

        parser_info_list = ["find_rels" for i in range(len(queries_list))]
        ex_rels = wiki_parser(parser_info_list, queries_list)
        ex_rels = list(set(ex_rels))
        ex_rels = [rel.split('/')[-1] for rel in ex_rels]

    #return question, ex_rels
    print("ex_rels :")
    print(ex_rels)

    rels_with_scores = rel_ranker.rank_rels(question, ex_rels)
    rels_with_scores = rels_with_scores[:rels_to_leave]
    return rels_with_scores

def t3_statement_df(query):
    assert type(query) == type([])
    assert len(query) == 3
    query_statement_df = search_triples_with_parse(
wiki_parser.document, query
).applymap(str).applymap(
    lambda x: search_entity_rep_by_lang_filter_in_db(x.split("/")[-1], wiki_entity_db, "zh") \
    if type(x) == type("") and x.startswith("http://www.wikidata.org/entity/Q") else x
)
    query_statement_df["p"] = query_statement_df["p"].map(
        lambda x: pid_zh_b_dict.get(x.split("/")[-1], "").split(" ") if x.split("/")[-1].startswith("P") else []
    )
    query_statement_df["s"] = query_statement_df["s"].map(
        lambda x: x if type(x) == type([]) else [str(x)]
    )
    query_statement_df["o"] = query_statement_df["o"].map(
        lambda x: x if type(x) == type([]) else [str(x)]
    )

    return query_statement_df

def fix_o(o, rm_char = ["\\"]):
    if not  o.startswith('"'):
        return o
    #print(o)
    assert o.startswith('"')
    num = []
    for i in range(len(o)):
        c = o[i]
        if c == '"':
            num.append(i)
    assert len(num) >= 2
    rm_num = num[1:-1]
    return "".join(
        map(lambda ii: o[ii], filter(lambda i: i not in rm_num and o[i] not in rm_char, range(len(o))))
    )

def py_dumpNtriple(
    subject, predicate, object_
):
    #### java rdfhdt dumpNtriple python format
    out =[]
    s0 = subject[0]
    if s0=='_' or s0 =='<':
        out.append(subject);
    else:
        out.append('<')
        out.append(subject)
        out.append('>')

    p0 = predicate[0]
    if p0=='<':
        out.append(' ')
        out.append(predicate)
        out.append(' ');
    else:
        out.append(" <")
        out.append(predicate)
        out.append("> ")

    o0 = object_[0]
    if o0=='"':
        #out.append(object_)
        ####
        #UnicodeEscape.escapeString(object.toString(), out);
        #out.append(json.dumps([object_])[1:-1])
        out.append(object_)
        out.append(" .\n");
    elif o0=='_' or o0=='<':
        out.append(object_)
        out.append(" .\n")
    else:
        out.append('<')
        out.append(object_)
        out.append("> .\n")
    return "".join(out)

def one_part_g_producer(one_part_string,
                       format_ = "nt"
                       ):
    from uuid import uuid1
    from rdflib import Graph

    tmp_f_name = "{}.{}".format(uuid1(), format_)
    with open(tmp_f_name, "w") as f:
        f.write(one_part_string)
    g = Graph()
    g.parse(tmp_f_name, format=format_)

    os.remove(tmp_f_name)
    return g

def drop_duplicates_by_col(df, on_col = "aug_sparql_query"):
    assert hasattr(df, "size")
    assert on_col in df.columns.tolist()
    req = []
    set_ = set([])
    for i, r in df.iterrows():
        if r[on_col] not in set_:
            set_.add(r[on_col])
            req.append(r)
    return pd.DataFrame(req)

def drop_duplicates_of_every_df(df):
    if not df.size:
        return df
    ori_columns = df.columns.tolist()
    df["hash"] = df.apply(lambda s: sha512(str(s.to_dict()).encode()).hexdigest(), axis = 1)
    req = []
    k_set = set([])
    for i, r in df.iterrows():
        if r["hash"] not in k_set:
            req.append(r.to_dict())
        k_set.add(r["hash"])
    return pd.DataFrame(req)[ori_columns]

def search_triples_with_parse(source ,query, return_df = True, skip_some_o = True, max_times = int(1e3)):
    assert hasattr(source, "search_triples")
    iter_, num = source.search_triples(*query)
    req = []
    for s, p, o in iter_:
        o = fix_o(o)
        if skip_some_o:
            if "\n" in o:
                continue
        nt_str = py_dumpNtriple(s, p, o)
        req.append(nt_str)
        if len(req) >= max_times:
            break
    g = one_part_g_producer("".join(req))
    if return_df:
        return pd.DataFrame(g.__iter__(), columns = ["s", "p", "o"])
    return g

def perm_top_sort(en_sent ,zh_perm_list, model, return_score = False):
    assert len(zh_perm_list) >= 1
    if len(zh_perm_list) == 1:
        return zh_perm_list[0]
    #### zh_perm_list length too big problem

    embedding = model.encode([en_sent] + zh_perm_list)

    sim_m = pytorch_cos_sim(embedding, embedding)
    sim_a = sim_m[0]
    if return_score:
        return sim_a.numpy()
    #### same top val 1
    max_index = np.argsort(sim_a.numpy()[1:])[-1]
    return zh_perm_list[max_index]

def syn_sim_on_list(sent, l):
    assert type(l) == type([])
    sim_df = pd.DataFrame(pd.Series(l).drop_duplicates().map(
        lambda x: (x,
        (synonyms.compare(sent, " ".join(re.findall(u"[\u4e00-\u9fa5]+", x)))\
         + (fuzz.ratio(sent, x) / 100.0)) / 2.0
        )
    ).values.tolist()
    )
    sim_df.columns = ["zh_info", "score"]
    sim_df = sim_df.sort_values(by = "score", ascending = False)
    return sim_df

def t3_statement_ranking(
    question,
    entity_list = ["http://www.wikidata.org/entity/Q42780"],
    property_list = ["http://www.wikidata.org/prop/direct/P131",
                    "http://www.wikidata.org/prop/direct/P150"
    ],
    generate_t3_func = lambda el, pl: pd.Series(list(product(el, pl))).map(
        lambda ep: [(ep[0], ep[1], ""), ("", ep[1], ep[0])]
    ).explode().dropna().drop_duplicates().tolist(),
    clf = b_clf,
    show_query = False,
    use_ranker = False,
):
    query_list = list(map(list ,generate_t3_func(entity_list, property_list)))
    #print(query_list)

    #df_list = list(map(t3_statement_df, query_list))
    df_list = []
    for ele in tqdm(query_list):
        if show_query:
            print(ele)
        df_list.append(t3_statement_df(ele))

    #return df_list

    assert len(query_list) == len(df_list)
    query_list_ = []
    df_list_ = []
    for i in range(len(query_list)):
        df = df_list[i]
        if hasattr(df, "size") and df.size > 0:
            query_list_.append(query_list[i])
            df_list_.append(df_list[i])
    assert len(query_list_) == len(df_list_)
    if len(query_list_) == 0:
        return None
    query_list = query_list_
    df_list = df_list_

    #print(len(df_list))
    #print("-" * 100)

    df_list = list(map(
        lambda df: df.applymap(
    lambda x: sorted(x, key = lambda y: fuzz.ratio(y, question), reverse = True)[0] if x else np.nan
    ).dropna()
    , df_list))

    query_list_ = []
    df_list_ = []
    for i in range(len(query_list)):
        df = df_list[i]
        if hasattr(df, "size") and df.size > 0:
            query_list_.append(query_list[i])
            df_list_.append(df_list[i])
    assert len(query_list_) == len(df_list_)
    if len(query_list_) == 0:
        return None
    query_list = query_list_
    df_list = df_list_

    #print(len(df_list))
    #print("-" * 100)

    req = []
    for i in range(len(query_list)):
        ele = df_list[i].copy()
        ele["cate"] = [tuple(query_list[i])] * len(ele)
        req.append(ele)

    abcd_df = pd.concat(req, axis = 0)
    abcd_df["key"] = abcd_df[["s", "p", "o"]].apply(lambda x: "".join(x.tolist()), axis = 1)

    ###return abcd_df

    if use_ranker:
        br_cls_s = RFR(b_clf,
            all_cate_list=abcd_df["key"].tolist(),sim_model=sim_model)
        rank_df = br_cls_s.produce_rank_df(question, br_cls_s.all_cate_list)
    else:
        rank_df = syn_sim_on_list(question, abcd_df["key"].tolist())
        rank_df = rank_df.rename(
            columns = {
                "score": "prob",
                "zh_info": "cate"
            }
        )

        br_cls_s = RFR(b_clf,
            all_cate_list=abcd_df["key"].tolist(),sim_model=sim_model)
        rank_df_ori = br_cls_s.produce_rank_df(question, br_cls_s.all_cate_list)

        rank_df = rank_df.reset_index().iloc[:, 1:]
        rank_df_ori = rank_df_ori.reset_index().iloc[:, 1:]

        print("rank_df :")
        print(rank_df)
        print("rank_df_ori :")
        print(rank_df_ori)
        print("merge :")
        print(pd.merge(rank_df, rank_df_ori, on = "cate"))
        print("-" * 100)

        merge_df = pd.merge(rank_df, rank_df_ori, on = "cate")
        cate_list = merge_df["cate"].tolist()
        prob_list = merge_df[["prob_x", "prob_y"]].max(axis = 1).tolist()
        rank_df = pd.concat([pd.Series(cate_list), pd.Series(prob_list)], axis = 1)
        rank_df.columns = ["cate", "prob"]

    abcd_df = pd.merge(rank_df[["cate", "prob"]].rename(
        columns = {
            "cate": "key"
        }
    ), abcd_df, on = "key").sort_values(by = "prob", ascending = False)

    abcd_uni_df = drop_duplicates_by_col(abcd_df, "cate")
    return abcd_uni_df

def choose_tmp_by_ranking(question,
    entity_list,
    tmp_conclusion_dict,
    tmp_generate_t3_func_dict,
    aug_func = max,
):
    assert type(question) == type("")
    assert type(entity_list) == type([])
    assert type(tmp_conclusion_dict) == type(dict())
    assert type(tmp_generate_t3_func_dict) == type(dict())
    assert len(tmp_conclusion_dict) == len(tmp_generate_t3_func_dict)
    assert set(tmp_conclusion_dict.keys()) == set(tmp_generate_t3_func_dict.keys())
    req = {}
    req_df_dict = {}
    for k in tmp_conclusion_dict.keys():
        till_list = tmp_conclusion_dict[k]
        assert type(till_list) == type([])
        if till_list:
            #print(k ,till_list)
            assert len(till_list[0]) == 3
        if not till_list:
            continue
        property_list = pd.Series(till_list).map(
            lambda t3: list(map(
                lambda x: x.format(t3[0]),
                ["http://www.wikidata.org/prop/direct/{}",
                "http://www.wikidata.org/prop/statement/{}",
                "http://www.wikidata.org/prop/{}",]
            ))
        ).explode().dropna().drop_duplicates().tolist()

        generate_t3_func = tmp_generate_t3_func_dict[k]
        assert callable(generate_t3_func)
        ranking_df = t3_statement_ranking(
            question = question,
            entity_list = entity_list,
            property_list = property_list,
            generate_t3_func = generate_t3_func
        )

        if not hasattr(ranking_df, "size") or ranking_df.size == 0:
            continue
        print("k ", k)
        print(ranking_df.head(3))

        ###assert ranking_df.shape[1] == 3
        assert "cate" in ranking_df.columns.tolist()
        assert "prob" in ranking_df.columns.tolist()
        score = aug_func(ranking_df["prob"].values.tolist())
        req[k] = score
        req_df_dict[k] = ranking_df
    for kk in tmp_conclusion_dict.keys():
        if kk not in req:
            req[kk] = -1
        if kk not in req_df_dict:
            req_df_dict[kk] = None
    ###return req
    best_tmp_cate = sorted(req.items(), key = lambda t2: t2[1], reverse = True)[0][0]
    assert best_tmp_cate in tmp_conclusion_dict
    ###best_ranking_df = req_df_dict[kk]
    best_ranking_df = req_df_dict[best_tmp_cate]

    a, b, c = best_tmp_cate, tmp_conclusion_dict[best_tmp_cate], best_ranking_df

    b_df = pd.DataFrame(b)

    if False:
        pass
    else:
        b_df[1] = b_df[1].map(
    lambda x:
    search_entity_rep_by_lang_filter_in_db(x.split("/")[-1], wiki_entity_db, "zh")\
    if type(x) == type("") and x.startswith("http://www.wikidata.org/entity/Q") else str(x)
).map(
    lambda x: x if type(x) != type("") else (re.findall('"(.+)"', x)[0] if len(x.split('"')) >= 3 else x)
)

    b_df.columns = ["pid", "entity", "score"]
    b_df = b_df.sort_values(by = "score", ascending = False)
    if c is None:
        c_dict = {"": 1.0}
    else:
        #c_dict = dict(c[["o", "prob"]].values.tolist())
        c_dict = dict(pd.DataFrame(pd.DataFrame(c.apply(
    lambda x:
    list(
    map(lambda i: (np.nan if x["cate"][i] else x[["s", "p", "o"]].tolist()[i], x["prob"]) ,range(len(x["cate"])))
    )
    , axis = 1).values.tolist()).values.reshape([-1]).tolist()).dropna().values.tolist())
        c_dict_ = {}
        for k, v in c_dict.items():
            c_dict_[str(k)] = v
        c_dict = c_dict_

    #return b_df, c_dict

    b_df_rewighted = pd.DataFrame(b_df.apply(
    lambda x:
        (x["pid"], x["entity"], x["score"] * \
        (max(map(lambda y:
    c_dict.get(y, min(c_dict.values())), x["entity"])) if x["entity"] and type(x["entity"]) == type([]) else \
    c_dict.get(x["entity"], min(c_dict.values())) if type(x["entity"]) == type("") else min(c_dict.values())
    )
    )
    , axis = 1
    ).values.tolist())
    b_df_rewighted.columns = b_df.columns.tolist()

    b_df_rewighted = b_df_rewighted.rename(
        columns = {"score": "multi_score"}
    )
    b_df_rewighted["score"] = b_df["score"].tolist()
    b_df_rewighted["rank_score"] = b_df_rewighted["multi_score"] / b_df_rewighted["score"]
    b_df_rewighted["max_score"] = b_df_rewighted[["score", "rank_score"]].apply(
        lambda x: x.max(), axis = 1
    )
    b_df_rewighted["sum_score"] = b_df_rewighted[["score", "rank_score"]].apply(
        lambda x: x.sum(), axis = 1
    )

    b_df_rewighted = b_df_rewighted.sort_values(by = "sum_score", ascending = False)

    return a, b_df_rewighted, best_ranking_df

def till_process_func(till_list):
    assert type(till_list) == type([])
    if not till_list:
        return till_list
    ele_length = list(map(len, till_list))
    assert len(set(ele_length)) == 1
    ele_length = ele_length[0]
    assert ele_length in [3, 4]
    if ele_length == 3:
        return till_list
    def filter_row(row_list):
        assert type(row_list) == type([])
        left = row_list[0]
        right = row_list[-1]
        mid = row_list[1:-1]
        assert mid
        if len(mid) == 1:
            return [left, mid[0], right]
        mid = list(filter(lambda x: not x.startswith("http://www.wikidata.org/prop/"), mid))
        assert mid
        return [left, mid[0], right]
    return list(map(filter_row, till_list))

def for_loop_detect(s, invalid_tag = "O-TAG", sp_token = "123454321"):
    assert type(s) == type(pd.Series())
    char_list = s.iloc[0]
    tag_list = s.iloc[1]
    assert len(char_list) == len(tag_list)
    req = defaultdict(list)
    pre_tag = ""
    for idx, tag in enumerate(tag_list):
        if tag == invalid_tag or tag != pre_tag:
            for k in req.keys():
                if req[k][-1] != invalid_tag:
                    req[k].append(sp_token)
            if tag != pre_tag and tag != invalid_tag:
                char = char_list[idx]
                req[tag].append(char)
        elif tag != invalid_tag:
            char = char_list[idx]
            req[tag].append(char)
        pre_tag = tag
    req = dict(map(lambda t2: (
        t2[0],
        list(
        filter(lambda x: x.strip() ,"".join(t2[1]).split(sp_token))
        )
                         ), req.items()))
    return req

def ner_entity_type_predict(question, id_slice_num = 5):
    assert type(question) == type("")
    question = question.replace(" ", "")
    ner_df = from_text_to_final(
    " ".join(list(question)),
    tokenizer,
    zh_model,
    label_list
)
    assert ner_df.shape[0] == len(question) + 2
    ### [UNK] filling
    ner_df[0] = ["[CLS]"] + list(question) + ["[SEP]"]
    et_dict = for_loop_detect(ner_df.T.apply(lambda x: x.tolist(), axis = 1))

    et_id_dict = dict(
map(lambda t2: (
    t2[0], list(map(lambda x: np.asarray(x).reshape([-1]).tolist() ,zh_linker_entities(
        list(map(lambda x: [x], t2[1]))
    )[0]))
) ,et_dict.items())
)
    ori_entity_ids = et_id_dict.get("E-TAG", [])
    ori_type_ids = et_id_dict.get("T-TAG", [])

    return ori_entity_ids, ori_type_ids, et_dict

def keyword_rule_filter(question_rm_et ,query_prob_dict):
    assert type(question_rm_et) == type("")
    assert type(query_prob_dict) == type({})
    if not question_rm_et.strip():
        return query_prob_dict
    def how_many_edit_filter(query_prob_dict):
        if not query_prob_dict:
            return query_prob_dict

        if 'SELECT (COUNT(?obj) AS ?value ) { wd:E1 wdt:R1 ?obj }'\
         not in reduce(lambda a, b : a + b ,query_prob_dict.values()):
            return query_prob_dict

        #### contain rm
        rm_contain_list = ["多大"]
        if any(map(lambda x: x in question_rm_et, rm_contain_list)):
            return dict(filter(
                lambda t2: 'SELECT (COUNT(?obj) AS ?value ) { wd:E1 wdt:R1 ?obj }' not in t2[1],
                query_prob_dict.items()
            ))
        return query_prob_dict

    apply_func_list = [how_many_edit_filter]
    query_prob_dict = deepcopy(query_prob_dict)
    for f_func in apply_func_list:
        query_prob_dict = f_func(query_prob_dict)

    return query_prob_dict

def tmp_type_predict(question, question_rm_et, b_clf = b_tmp_clf, consider_tmp_prob = 0.2,
    show_query_prob_dict = False
):
    assert type(question) == type("")
    assert type(question_rm_et) == type("")
    prob_query_dict = tmp_from_text_to_final(question, cls_model = b_clf, sim_model = sim_model, return_query=True,
                      return_prob = True,
                      )
    assert type(prob_query_dict) == type({})
    query_prob_dict = dict(filter(lambda t2: t2[0] >= consider_tmp_prob, prob_query_dict.items()))

    if show_query_prob_dict:
        print("before :" ,query_prob_dict)

    query_prob_dict = keyword_rule_filter(question_rm_et ,query_prob_dict)

    if show_query_prob_dict:
        print("after :" ,query_prob_dict)

    query_list = list(map(lambda tt2: tt2[1] ,sorted(query_prob_dict.items(), key = lambda t2: t2[0], reverse = True)))
    query_list = reduce(lambda a, b: a + b, query_list) if query_list else []
    query_list = list(map(lambda x: x.strip(), query_list))

    tmp_func_7 = lambda el, pl: pd.Series(list(product(el, pl))).map(
            lambda ep: [(ep[0], ep[1], "")]
        ).explode().dropna().drop_duplicates().tolist()
    tmp_func_8 = lambda el, pl: pd.Series(list(product(el, pl))).map(
            lambda ep: [("", ep[1], ep[0])]
        ).explode().dropna().drop_duplicates().tolist()

    tmp_generate_t3_func_dict = {
        sparql_queries_df.iloc[0].to_dict()["query_template"]: tmp_func_7,
        sparql_queries_df.iloc[1].to_dict()["query_template"]: tmp_func_7,
        sparql_queries_df.iloc[2].to_dict()["query_template"]: tmp_func_7,
        sparql_queries_df.iloc[4].to_dict()["query_template"]: tmp_func_7,
        sparql_queries_df.iloc[7].to_dict()["query_template"]: tmp_func_7,
    }

    return dict(filter(lambda t2: t2[0] in query_list, tmp_generate_t3_func_dict.items()))

def property_df_rep_disambiguation(question ,property_df,
rep_col = "entity", property_col = "pid", order_col = "sum_score",
    use_emb = True
):
    print("do disamb in {}".format(question))
    property_df = drop_duplicates_of_every_df(property_df.sort_values(by = order_col, ascending = False))
    if not hasattr(property_df, "size") or property_df.size == 0:
        return property_df
    property_df_list = []
    for pid, df in property_df.groupby(property_col):
        s = df[rep_col][
            df[rep_col].map(bool)
        ]
        if not s.size:
            continue
        max_score = df[order_col].max()
        if use_emb:
            req_df = pd.DataFrame(s.map(
            lambda x: (x,
            ((max(
            perm_top_sort(question, x, sim_model, return_score = True)[1:]
            )) if len(x) >= 2 else synonyms.compare(x[0], question)) if type(x) == type([]) \
            else 1.0)
            ).values.tolist())
        else:
            req_df = pd.DataFrame(s.map(
            lambda x: (x,
            max(map(lambda y: synonyms.compare(y, question), x)) if type(x) == type([]) \
            else 1.0
            )
        ).values.tolist())
        req_df[2] = req_df[0].map(lambda x: -1 * len(x))
        rep_list = req_df.sort_values(by = [1, 2], ascending = False).iloc[:, 0].tolist()
        df = df[
            df[rep_col].map(
                lambda x: x in rep_list
            )
        ]
        df = df.iloc[:1, :]
        df[rep_col] = [rep_list] * len(df)
        df[order_col] = max_score
        property_df_list.append(df)
    req = pd.concat(property_df_list, axis = 0).sort_values(by = order_col, ascending = False)[
        ['pid',
 'entity',
 'multi_score',
 'score',
 'rank_score',
 'max_score',
 'sum_score']
    ]
    req["pid_in_cnt"] = req["pid"].map(
        lambda x: int(any(map(lambda y: y in question,
            list(filter(lambda xxx: xxx ,map(lambda xx: xx.strip() ,pid_zh_b_dict[x].split(" "))))\
             if x in pid_zh_b_dict else ["1-2-3-4-5-6-7-8-9-0"]
        )))
    )
    req = req.sort_values(by = ["pid_in_cnt", order_col], ascending = False)
    return req


def do_search(question, in_string_entity_overload_dict = {

}, do_property_disamb = True):
    ##### ner scope
    ori_entity_ids, ori_type_ids, et_dict = ner_entity_type_predict(question)

    question_rm_et = question
    assert type(et_dict) == type(dict())
    for k, v in et_dict.items():
        assert type(v) == type([])
        for vv in sorted(v, key = len, reverse = True):
            if vv:
                question_rm_et = question_rm_et.replace(vv, "")

    print("ori_entity_ids :")
    print(ori_entity_ids)

    overload_entityid_list = list(map(lambda tt2: tt2[1], filter(
    lambda t2: t2[0] in question
    , in_string_entity_overload_dict.items())))
    overload_entityid_list = list(set(overload_entityid_list).difference(set(
    reduce(lambda a, b: a + b, ori_entity_ids) if ori_entity_ids else ori_entity_ids
    ))) \
    if ori_entity_ids else overload_entityid_list

    if overload_entityid_list:
        assert overload_entityid_list[0].startswith("Q")

    print("overload_entityid_list :")
    print(overload_entityid_list)

    if overload_entityid_list and ori_entity_ids:
        need_extend_index = None
        zh_rep_list = list(map(
        lambda i:
        list(map(lambda x:
        search_entity_rep_by_lang_filter_in_db(x[1:] if x.startswith("XQ") else x, wiki_entity_db, "zh")
        , ori_entity_ids[i])),
        range(len(ori_entity_ids))))
        assert len(zh_rep_list) == len(ori_entity_ids)
        zh_rep_list = list(map(lambda x: list(set(reduce(lambda a, b: a + b, x))) if x else x,
        zh_rep_list))
        zh_rep_list = list(map(lambda x: x if x else [""], zh_rep_list))
        assert len(zh_rep_list) == len(ori_entity_ids)
        for ele in zh_rep_list:
            assert type(ele) == type([])
            assert ele

        in_string_entity_overload_dict_rev = dict(map(lambda t2: t2[::-1] ,
        in_string_entity_overload_dict.items()))
        for ele in overload_entityid_list:
            assert ele in in_string_entity_overload_dict_rev

        overload_entityid_index_dict = dict(map(
        lambda id:
        (id,

        sorted(map(
        lambda t2: (t2[0] ,

        max(map(lambda x:
        fuzz.ratio(x, in_string_entity_overload_dict_rev[id])
        ,t2[1]))

        )
        , enumerate(zh_rep_list)
        ),
        key = lambda tt2: tt2[1], reverse = True
        )[0][0] if sorted(map(
        lambda t2: (t2[0] ,

        max(map(lambda x:
        fuzz.ratio(x, in_string_entity_overload_dict_rev[id])
        ,t2[1]))

        )
        , enumerate(zh_rep_list)
        ),
        key = lambda tt2: tt2[1], reverse = True
        )[0][1] >= 50.0 else -1

        )
        , overload_entityid_list))

        print("overload_entityid_index_dict :")
        print(overload_entityid_index_dict)
        print("-" * 100)

        for k, v in overload_entityid_index_dict.items():
            assert k in overload_entityid_list
            assert v < len(ori_entity_ids) and v >= -1

        overload_entityid_index_dict_rev = defaultdict(list)
        for k, v in overload_entityid_index_dict.items():
            overload_entityid_index_dict_rev[v].append(k)

        ori_entity_ids_ = []
        for i, inner_list in enumerate(ori_entity_ids):
            ele = deepcopy(inner_list)
            if i in overload_entityid_index_dict_rev:
                assert type(overload_entityid_index_dict_rev[i]) == type([])
                ele = (overload_entityid_index_dict_rev[i] + ele)[:len(ele)]
            ori_entity_ids_.append(ele)

        if -1 in overload_entityid_index_dict_rev:
            ori_entity_ids_.append(overload_entityid_index_dict_rev[-1])

        assert len(ori_entity_ids_) >= len(ori_entity_ids) and len(ori_entity_ids_) in [0, 1, 2]
        ori_entity_ids = deepcopy(ori_entity_ids_)

    ori_entity_ids = list(filter(lambda x:
    list(filter(lambda y: y.startswith("Q") ,x))
     ,ori_entity_ids))

    if not ori_entity_ids:
        return None

    print("ori_entity_ids :")
    print(ori_entity_ids)

    ###### tmp classifier scope
    tmp_generate_t3_func_dict = tmp_type_predict(question, question_rm_et)

    tmp_generate_t3_func_dict = dict(filter(
        lambda t2: len(ori_entity_ids) != 1 if t2[0] in [
            'SELECT ?value WHERE { wd:E1 p:R1 ?s . ?s ps:R1 wd:E2 . ?s ?p ?value }'
        ] else True
    , tmp_generate_t3_func_dict.items()))

    if not tmp_generate_t3_func_dict:
        tmp_func_7 = lambda el, pl: pd.Series(list(product(el, pl))).map(
            lambda ep: [(ep[0], ep[1], "")]
        ).explode().dropna().drop_duplicates().tolist()

        tmp_generate_t3_func_dict = {
            sparql_queries_df.iloc[7].to_dict()["query_template"]: tmp_func_7,
        }

    query_list = sparql_queries_df["query_template"].tolist()
    assert len(set(tmp_generate_t3_func_dict.keys()).intersection(set(query_list))) == len(tmp_generate_t3_func_dict)

    ###### ranking scope
    tmp_conclusion_dict = {}
    for query, func in tmp_generate_t3_func_dict.items():
        index = query_list.index(query)
        till = deepcopy(query_parser_bu(question,
        sparql_queries_df.iloc[index].to_dict(),
        sparql_queries_df.iloc[index]["entities_and_types_select"]
                                ,
                              ori_entity_ids, ori_type_ids))
        till = till_process_func(till)
        tmp_conclusion_dict[query] = till

    assert len(tmp_conclusion_dict) == len(tmp_generate_t3_func_dict)
    #return ori_entity_ids, tmp_conclusion_dict, tmp_generate_t3_func_dict
    a, b, c = choose_tmp_by_ranking(
        question,
        list(map(
            lambda x: "http://www.wikidata.org/entity/{}".format(x)
            ,reduce(lambda a, b: a + b, ori_entity_ids)
            )),
        tmp_conclusion_dict,
        tmp_generate_t3_func_dict,
        aug_func = max,
    )
    if do_property_disamb:
        if not ori_type_ids:
            b = property_df_rep_disambiguation(question, b)
        else:
            type_id_zh_rep_list = list(map(lambda l:
            list(
            map(lambda id:
                search_entity_rep_by_lang_filter_in_db(id, wiki_entity_db, "zh") \
                if id.startswith("Q") else pid_zh_b_dict.get(id, "").split(" ")
                , l))
                                      , ori_type_ids)
                                     )

            print(type_id_zh_rep_list)
            type_id_zh_rep_list_ = []
            for inner_list in type_id_zh_rep_list:
                for ele in inner_list:
                    ele = list(filter(lambda x: x in question, ele))
                    for x in ele:
                        if x not in type_id_zh_rep_list_ and x in question:
                            type_id_zh_rep_list_.append(x)
            type_id_zh_rep_list = type_id_zh_rep_list_
            for ele in type_id_zh_rep_list:
                assert ele.strip()
            if not type_id_zh_rep_list:
                b = property_df_rep_disambiguation(question, b)
            else:
                b = property_df_rep_disambiguation(" ".join(type_id_zh_rep_list), b)

    return a, b, c


will del in if :
{'class': 'logging.FileHandler', 'level': 'DEBUG', 'formatter': 'default', 'filename': '~/.deeppavlov/log.log'}
skip del
else :
set root
[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/Users/svjack/opt/anaconda3/envs/py377/lib/python3.7/site-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/Users/svjack/opt/anaconda3/envs/py377/lib/python3.7/site-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/Users/svjack/opt/anaconda3/envs/py377/lib/python3.7/site-packages/synonyms/data/words.vector.gz] ...


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

In [2]:
question = "宁波在哪个省？"
a, b, c = do_search(question)
b

order_dict_for_merge :
{}
srtd_with_ratios :
[(3988110, 'Q42780', 0, 1, 100.0, 6), (1761264, 'Q913727', 0, 1, 80.0, 5), (2758402, 'Q10945964', 0, 1, 80.0, 4), (622463, 'Q1017158', 0, 1, 80.0, 3), (2727634, 'Q11096009', 0, 1, 80.0, 3), (2617780, 'Q10945988', 0, 1, 66.66666666666667, 5), (141615, 'Q1000022', 0, 1, 66.66666666666667, 5), (2120205, 'Q2925232', 0, 1, 66.66666666666667, 4), (2983162, 'Q11091988', 0, 1, 66.66666666666667, 4), (4705503, 'Q4856166', 0, 1, 66.66666666666667, 4), (4305239, 'Q7038655', 0, 1, 66.66666666666667, 4), (940151, 'Q873076', 0, 1, 66.66666666666667, 4), (1721468, 'Q7038661', 0, 1, 66.66666666666667, 4), (2846115, 'Q13902059', 0, 1, 66.66666666666667, 4), (5453426, 'Q10946037', 0, 1, 66.66666666666667, 4), (348142, 'Q10945982', 0, 1, 66.66666666666667, 4), (5975191, 'Q200810', 0, 1, 66.66666666666667, 3), (1774752, 'Q24839374', 0, 1, 66.66666666666667, 3), (2935830, 'Q15904161', 0, 1, 66.66666666666667, 3), (4892348, 'Q10132577', 0, 1, 66.66666666666667, 3

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q42780', 'Q913727', 'Q10945964', 'Q1017158', 'Q11096009']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q11096009', 'forw', 'direct'), ('Q1017158', 'forw', 'direct'), ('Q10945964', 'forw', 'direct'), ('Q913727', 'forw', 'direct'), ('Q42780', 'forw', 'direct')]
ex_rels :
['P31', 'P349', 'P910', 'P982', 'P137', 'P646', 'P569', 'P81', 'P18', 'P268', 'P281', 'P395', 'P2046', 'P150', 'P373', 'P227', 'P134', 'P131', 'P473', 'P856', 'P948', 'P1464', 'P442', 'P833', 'P1792', 'P935', 'P2044', 'P17', 'P625', 'P1082', 'P1566', 'P1378', 'P197', 'P296', 'P421', 'P998', 'P1448', 'P190', 'P242', 'P37']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P131,"[[浙江省, 浙, 浙江], [海曙区, 海曙]]",0.821402,0.992421,0.827675,0.992421,1.820096,1
1,P150,"[[镇海区], [奉化区, 奉化, 奉化縣], [慈溪市, 慈溪, 慈溪县, 慈谿, 慈谿縣...",0.727966,0.972107,0.748854,0.972107,1.720961,0
4,P2046,[+9816],0.484225,0.824421,0.587352,0.824421,1.411773,0
5,P17,"[[中华人民共和国, 中国, 中國, 中華人民共和國, 共产中国, 新中国]]",0.452057,0.807789,0.559622,0.807789,1.367411,0
6,P625,"[Point(121.54916666667 29.875), Point(121.5326...",0.430882,0.851015,0.506316,0.851015,1.357330,0
16,P190,"[[益田市], [鲁昂, Rouen, 盧昂], [奧克蘭都會區, Auckland, 奧克...",0.278024,0.388582,0.715483,0.715483,1.104065,0
17,P421,[[UTC+8]],0.230631,0.715136,0.322500,0.715136,1.037636,0
18,P197,"[[宁波东站], [庄桥站]]",0.202390,0.444816,0.454998,0.454998,0.899814,0
19,P833,"[[宁波火车站, 铁路南站站]]",0.171585,0.513475,0.334164,0.513475,0.847639,0
25,P2044,[+150],0.074995,0.242219,0.309618,0.309618,0.551837,0


In [3]:
question = "美国的货币是什么？"
a, b, c = do_search(question)
b

order_dict_for_merge :
{'Q30': 0}
srtd_with_ratios :
[(872101, 'Q30', 0, 1, 100.0, 9), (1801253, 'Q19093', -1, 1, 100.0, 4), (975820, 'Q48229', -1, 1, 100.0, 3), (5338499, 'Q860546', -1, 1, 80.0, 8), (2323822, 'Q468996', -1, 1, 80.0, 8), (5022621, 'Q3089216', -1, 1, 80.0, 8), (1052075, 'Q220713', -1, 1, 80.0, 6), (5058892, 'Q1054376', -1, 1, 80.0, 4), (4845510, 'Q469382', -1, 1, 80.0, 3), (5643779, 'Q214100', -1, 1, 80.0, 3), (442117, 'Q338242', -1, 1, 80.0, 3), (411282, 'Q173941', -1, 1, 66.66666666666667, 14), (1812313, 'Q668687', -1, 1, 66.66666666666667, 13), (2537656, 'Q36949', -1, 1, 66.66666666666667, 13), (2502399, 'Q275120', -1, 1, 66.66666666666667, 11), (4440575, 'Q557081', -1, 1, 66.66666666666667, 11), (3850847, 'Q550678', -1, 1, 66.66666666666667, 11), (4709437, 'Q33659', -1, 1, 66.66666666666667, 10), (4334503, 'Q37079', -1, 1, 66.66666666666667, 10), (2071784, 'Q35657', -1, 1, 66.66666666666667, 10), (625774, 'Q39681', -1, 1, 66.66666666666667, 8), (4472091, 'Q2168496',

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q30', 'Q19093', 'Q48229', 'Q860546', 'Q468996']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q860546', 'forw', 'direct'), ('Q30', 'forw', 'direct'), ('Q19093', 'forw', 'direct'), ('Q48229', 'forw', 'direct'), ('Q468996', 'forw', 'direct')]
ex_rels :
['P961', 'P1813', 'P830', 'P3479', 'P349', 'P1151', 'P910', 'P982', 'P2959', 'P3417', 'P212', 'P18', 'P1005', 'P856', 'P3762', 'P846', 'P1792', 'P225', 'P2299', 'P2853', 'P6', 'P898', 'P1281', 'P94', 'P50', 'P1906', 'P36', 'P163', 'P1070', 'P463', 'P1333', 'P2633', 'P3422', 'P242', 'P2134', 'P906', 'P3222', 'P402', 'P1843', 'P3183', 'P141', 'P1313', 'P214', 'P1943', 'P360', 'P1842', 'P373', 'P984', 'P2852', 'P2258', 'P1451', 'P1332', 'P244', 'P1421', 'P832', 'P474', 'P158', 'P3221', 'P1740', 'P1325', 'P421', 'P998', 'P1017', 'P495', 'P37', 'P31', 'P1296', 'P1343', 'P47', 'P960', 'P1081', 'P30', 'P2184', 'P1465', 'P418', 'P364

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P38,"[[美元, 美金]]",0.875863,0.991404,0.883457,0.991404,1.874861,1
1,P1325,[https://www.data.gov/],0.804805,0.942852,0.853586,0.942852,1.796438,0
2,P495,"[[德国, 德國, 德意志, 德意志联邦, 德意志联邦共和国, 德意志聯邦, 德意志聯邦共和國]]",0.678130,0.775325,0.874639,0.874639,1.649964,0
3,P364,[[德语]],0.654808,0.702632,0.931936,0.931936,1.634568,0
4,P37,"[[英语, en, 英文, 英語]]",0.611448,0.879106,0.695534,0.879106,1.574640,0
5,P1843,[American lotus],0.441670,0.596518,0.740413,0.740413,1.336931,0
6,P2184,"[[美国历史, 美史, 美國歷史]]",0.430763,0.612836,0.702900,0.702900,1.315736,0
7,P2884,[+120],0.364467,0.700731,0.520124,0.700731,1.220855,0
8,P141,"[[无危物种, 低关注度物种, 低關注度物種, 无危, 最低關注物種, 無危]]",0.351570,0.737822,0.476497,0.737822,1.214318,0
9,P948,[http://commons.wikimedia.org/wiki/Special:Fil...,0.347527,0.583013,0.596088,0.596088,1.179101,0


In [4]:
question = "川普什么时候出生？"
a, b, c = do_search(question)
b

order_dict_for_merge :
{}
srtd_with_ratios :
[(2622649, 'Q22686', 0, 1, 100.0, 13), (4379647, 'Q408352', 0, 1, 100.0, 4), (5760110, 'Q2462124', 0, 1, 66.66666666666667, 9), (6158228, 'Q635789', 0, 1, 66.66666666666667, 3), (2039982, 'Q24036505', 0, 1, 66.66666666666667, 3), (1303729, 'Q7847772', 0, 1, 66.66666666666667, 3), (1302237, 'Q3713655', 0, 1, 50.0, 5), (6107819, 'Q844324', 0, 1, 50.0, 5), (5114258, 'Q3731533', 0, 1, 50.0, 4), (5625045, 'Q239411', 0, 1, 50.0, 4), (1128719, 'Q242351', 0, 1, 50.0, 4), (3029259, 'Q432473', 0, 1, 44.44444444444444, 3), (2304714, 'Q287589', 0, 1, 40.0, 4), (2160566, 'Q546221', 0, 1, 40.0, 3), (822553, 'Q2456392', 0, 1, 36.36363636363637, 4), (3936018, 'Q14410370', 0, 1, 30.76923076923077, 4), (4903478, 'Q8873894', 0, 1, 26.66666666666667, 4), (5810695, 'Q15292086', 0, 1, 25.0, 4), (345114, 'Q28554276', 0, 1, 23.529411764705888, 4), (2930995, 'Q27977945', 0, 1, 19.999999999999996, 4), (3017699, 'Q27824000', 0, 1, 19.999999999999996, 4)]
-------------

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q22686', 'Q408352', 'Q2462124', 'Q635789', 'Q24036505']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q22686', 'forw', 'direct'), ('Q408352', 'forw', 'direct'), ('Q2462124', 'forw', 'direct'), ('Q24036505', 'forw', 'direct'), ('Q635789', 'forw', 'direct')]
ex_rels :
['P1284', 'P3479', 'P19', 'P349', 'P1151', 'P910', 'P1003', 'P1559', 'P3417', 'P3373', 'P1185', 'P3344', 'P26', 'P569', 'P18', 'P1890', 'P2013', 'P1971', 'P937', 'P159', 'P512', 'P1005', 'P1477', 'P2435', 'P856', 'P1283', 'P1280', 'P154', 'P551', 'P1454', 'P1263', 'P40', 'P1472', 'P898', 'P172', 'P94', 'P2218', 'P2190', 'P2003', 'P271', 'P553', 'P463', 'P2397', 'P1430', 'P906', 'P3222', 'P127', 'P1258', 'P3183', 'P740', 'P1220', 'P2638', 'P27', 'P140', 'P1411', 'P1819', 'P214', 'P1839', 'P2048', 'P373', 'P3509', 'P1368', 'P2949', 'P1015', 'P1449', 'P3430', 'P25', 'P1953', 'P691', 'P1424', 'P2605', 'P1266', 'P3

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

[Synonyms] not exist in w2v model: 皇后區


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Synonyms] not exist in w2v model: 曼哈頓
[Synonyms] not exist in w2v model: 2017
[Synonyms] not exist in w2v model: 美國
[Synonyms] not exist in w2v model: 總統
[Synonyms] not exist in w2v model: 就職典禮
[Synonyms] not exist in w2v model: 2016
[Synonyms] not exist in w2v model: 唐納德
[Synonyms] not exist in w2v model: 競選
[Synonyms] not exist in w2v model: 美國
[Synonyms] not exist in w2v model: 總統


,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P569,[1946-06-14T00:00:00Z],0.571947,0.934645,0.611940,0.934645,1.546585,0
1,P1477,[Donald John Trump],0.323182,0.692781,0.466500,0.692781,1.159281,0
2,P571,"[1989-01-01T00:00:00Z, 1923-01-01T00:00:00Z]",0.174494,0.842636,0.207081,0.842636,1.049718,0
4,P1971,[+5],0.200211,0.619847,0.323000,0.619847,0.942847,0
5,P19,[[牙买加医院]],0.162826,0.362216,0.449526,0.449526,0.811742,0
6,P17,"[[美国, US, USA, 美利坚合众国, 美國, 花旗国]]",0.103828,0.437821,0.237147,0.437821,0.674968,0
7,P734,[[特朗普]],0.097299,0.460210,0.211423,0.460210,0.671633,0
8,P551,"[[白宮, 白屋, 美國白宮], [棕榈滩, 棕櫚灘], [杜林普大廈, 川普大廈, 川普大...",0.091262,0.214931,0.424611,0.424611,0.639542,0
9,P21,"[[男性, 男]]",0.086212,0.271656,0.317357,0.317357,0.589013,0
10,P793,"[[2017年美國總統就職典禮], [2016年唐納德·川普競選美國總統]]",0.055776,0.276907,0.201423,0.276907,0.478330,0


In [5]:
question = "川端康成出生在哪里?"
a, b, c = do_search(question)
b

order_dict_for_merge :
{}
srtd_with_ratios :
[(5300634, 'Q43736', 0, 1, 100.0, 3), (5935043, 'Q1576163', 0, 1, 72.72727272727273, 4), (503167, 'Q11215449', 0, 1, 47.05882352941176, 4), (2430623, 'Q9615828', 0, 1, 31.999999999999996, 4)]
----------------------------------------------------------------------------------------------------
order_dict_for_merge :
{'Q14819852': 0}
srtd_with_ratios :
[(5011737, 'Q14819852', 0, 1, 100.0, 3), (2356615, 'P569', -1, 1, 80.0, 7), (2036358, 'Q1322263', -1, 1, 80.0, 4), (5764613, 'Q2643488', -1, 1, 80.0, 3), (5704886, 'P1477', -1, 1, 66.66666666666667, 6), (2018446, 'Q2914808', -1, 1, 66.66666666666667, 4), (704867, 'Q21821348', -1, 1, 66.66666666666667, 3), (5276205, 'Q225916', -1, 1, 66.66666666666667, 3), (4834463, 'Q83900', -1, 1, 57.14285714285714, 5), (3364882, 'P1464', -1, 1, 57.14285714285714, 5), (4872921, 'Q604971', -1, 1, 57.14285714285714, 5), (1866991, 'Q10896659', -1, 1, 57.14285714285714, 3), (2990017, 'Q1146100', -1, 1, 50.0, 19), (1

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q43736', 'Q1576163', 'Q11215449', 'Q9615828']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q9615828', 'forw', 'direct'), ('Q43736', 'forw', 'direct'), ('Q1576163', 'forw', 'direct'), ('Q11215449', 'forw', 'direct')]
ex_rels :
['P31', 'P1296', 'P906', 'P1343', 'P106', 'P213', 'P19', 'P349', 'P396', 'P1315', 'P1309', 'P1003', 'P1559', 'P1814', 'P910', 'P3417', 'P646', 'P27', 'P26', 'P1411', 'P214', 'P18', 'P1207', 'P268', 'P569', 'P571', 'P1412', 'P21', 'P1006', 'P269', 'P3348', 'P570', 'P166', 'P1005', 'P3569', 'P373', 'P69', 'P227', 'P2435', 'P949', 'P1015', 'P1196', 'P866', 'P20', 'P800', 'P3430', 'P138', 'P1417', 'P409', 'P2188', 'P691', 'P2019', 'P2605', 'P1472', 'P1266', 'P103', 'P2639', 'P345', 'P735', 'P244', 'P17', 'P1273', 'P136', 'P648', 'P2163', 'P1289', 'P271', 'P950', 'P3188', 'P535']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P19,"[[大阪市, 大坂, 大阪]]",0.650168,0.997189,0.652000,0.997189,1.649189,0
1,P20,[[逗子市]],0.417532,0.755031,0.553000,0.755031,1.308031,0
2,P569,[1899-06-11T00:00:00Z],0.359537,0.725367,0.495662,0.725367,1.221029,0
3,P17,"[[日本, 日本国, 日本國]]",0.153549,0.587937,0.261167,0.587937,0.849103,0
4,P570,[1972-04-16T00:00:00Z],0.098117,0.224999,0.436079,0.436079,0.661078,0
5,P1196,[[自殺]],0.106249,0.303809,0.349722,0.349722,0.653532,0
6,P21,"[[男性, 男]]",0.088187,0.224512,0.392794,0.392794,0.617306,0
7,P571,[1974-01-01T00:00:00Z],0.069357,0.244215,0.284000,0.284000,0.528215,0
8,P138,[[川端康成]],0.009398,0.025719,0.365414,0.365414,0.391133,0
9,P27,"[[日本, 日本国, 日本國]]",0.012935,0.049528,0.261167,0.261167,0.310695,0


In [6]:
question = "台北面积是多少？"
a, b, c = do_search(question)
b

order_dict_for_merge :
{}
srtd_with_ratios :
[(1915528, 'Q1867', 0, 1, 100.0, 7), (5413717, 'Q712937', 0, 1, 80.0, 13), (1022175, 'Q244898', 0, 1, 80.0, 12), (5603527, 'Q10914394', 0, 1, 80.0, 5), (502278, 'Q11063980', 0, 1, 80.0, 5), (5216053, 'Q698122', 0, 1, 80.0, 5), (2995615, 'Q697946', 0, 1, 80.0, 4), (4742001, 'Q15943758', 0, 1, 80.0, 3), (441296, 'Q7231276', 0, 1, 80.0, 3), (3358772, 'Q10914410', 0, 1, 80.0, 3), (967080, 'Q10927172', 0, 1, 66.66666666666667, 29), (950352, 'Q7676551', 0, 1, 66.66666666666667, 22), (6173685, 'Q44856', 0, 1, 66.66666666666667, 20), (5393454, 'Q697974', 0, 1, 66.66666666666667, 19), (5781205, 'Q6424882', 0, 1, 66.66666666666667, 19), (3182420, 'Q3972257', 0, 1, 66.66666666666667, 19), (2831322, 'Q5972821', 0, 1, 66.66666666666667, 17), (1839714, 'Q10914117', 0, 1, 66.66666666666667, 17), (1218725, 'Q707456', 0, 1, 66.66666666666667, 16), (1678795, 'Q6978828', 0, 1, 66.66666666666667, 16), (3113188, 'Q7676312', 0, 1, 66.66666666666667, 15), (3011958

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q1867', 'Q712937', 'Q244898', 'Q10914394', 'Q11063980']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q10914394', 'forw', 'direct'), ('Q712937', 'forw', 'direct'), ('Q244898', 'forw', 'direct'), ('Q1867', 'forw', 'direct'), ('Q11063980', 'forw', 'direct')]
ex_rels :
['P31', 'P961', 'P830', 'P47', 'P3591', 'P501', 'P1843', 'P960', 'P349', 'P910', 'P982', 'P2959', 'P3417', 'P646', 'P2184', 'P1376', 'P214', 'P18', 'P105', 'P1465', 'P1997', 'P268', 'P269', 'P281', 'P194', 'P2046', 'P150', 'P373', 'P227', 'P131', 'P473', 'P856', 'P948', 'P1464', 'P949', 'P41', 'P846', 'P1705', 'P225', 'P1792', 'P171', 'P3088', 'P6', 'P935', 'P2044', 'P17', 'P244', 'P94', 'P625', 'P1082', 'P1566', 'P158', 'P36', 'P1070', 'P1740', 'P421', 'P998', 'P300', 'P190', 'P242', 'P901']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

question, order_info: 台北面积是多少？, order_info(variable=None, sorting_order=None)

...

[Synonyms] not exist in w2v model: 區
[Synonyms] not exist in w2v model: 萬華區
[Synonyms] not exist in w2v model: 新莊
[Synonyms] not exist in w2v model: 區
[Synonyms] not exist in w2v model: 區
[Synonyms] not exist in w2v model: 北投區
[Synonyms] not exist in w2v model: 大安區
[Synonyms] not exist in w2v model: 士林區
[Synonyms] not exist in w2v model: 大同區
[Synonyms] not exist in w2v model: 三峽
[Synonyms] not exist in w2v model: 區


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Synonyms] not exist in w2v model: 南港區
[Synonyms] not exist in w2v model: 松山區
[Synonyms] not exist in w2v model: 信義
[Synonyms] not exist in w2v model: 區
[Synonyms] not exist in w2v model: 中和區
[Synonyms] not exist in w2v model: 山區
[Synonyms] not exist in w2v model: 區
[Synonyms] not exist in w2v model: 城區
[Synonyms] not exist in w2v model: 雙溪
[Synonyms] not exist in w2v model: 區
[Synonyms] not exist in w2v model: 鶯歌
[Synonyms] not exist in w2v model: 區


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P2046,[+271799700],0.681225,0.999129,0.681818,0.999129,1.680948,1
1,P625,"[Point(121.445833 25.011111), Point(121.633333...",0.604967,0.981869,0.616139,0.981869,1.598007,0
3,P421,[[UTC+8]],0.396120,0.994383,0.398357,0.994383,1.392740,0
4,P150,"[[林口區, 坪頂, 樹林口], [深坑區, 深坑仔, 簪纓], [中山區, 中山区], [...",0.390671,0.454252,0.860030,0.860030,1.314283,0
6,P2044,[+10],0.352009,0.685297,0.513660,0.685297,1.198956,0
7,P17,"[[中華民國, ROC, 中华台北, 中华民国, 中華台北, 台湾, 台灣, 民国, 民國]]",0.155436,0.407078,0.381833,0.407078,0.788912,0
8,P131,"[[中華民國, ROC, 中华台北, 中华民国, 中華台北, 台湾, 台灣, 民国, 民國]]",0.118409,0.310107,0.381833,0.381833,0.691940,0
48,P1082,"[+2655515, +4054467]",0.074724,0.214108,0.349000,0.349000,0.563108,0
50,P1843,"[臺北薹, 銳果薹]",0.046964,0.156285,0.300500,0.300500,0.456785,0
51,P105,"[[种, 物種, 種]]",0.037858,0.162479,0.233000,0.233000,0.395479,0


In [7]:
question = "台北面积有多大？"
a, b, c = do_search(question)
b

order_dict_for_merge :
{}
srtd_with_ratios :
[(1915528, 'Q1867', 0, 1, 100.0, 7), (5413717, 'Q712937', 0, 1, 80.0, 13), (1022175, 'Q244898', 0, 1, 80.0, 12), (5603527, 'Q10914394', 0, 1, 80.0, 5), (502278, 'Q11063980', 0, 1, 80.0, 5), (5216053, 'Q698122', 0, 1, 80.0, 5), (2995615, 'Q697946', 0, 1, 80.0, 4), (4742001, 'Q15943758', 0, 1, 80.0, 3), (441296, 'Q7231276', 0, 1, 80.0, 3), (3358772, 'Q10914410', 0, 1, 80.0, 3), (967080, 'Q10927172', 0, 1, 66.66666666666667, 29), (950352, 'Q7676551', 0, 1, 66.66666666666667, 22), (6173685, 'Q44856', 0, 1, 66.66666666666667, 20), (5393454, 'Q697974', 0, 1, 66.66666666666667, 19), (5781205, 'Q6424882', 0, 1, 66.66666666666667, 19), (3182420, 'Q3972257', 0, 1, 66.66666666666667, 19), (2831322, 'Q5972821', 0, 1, 66.66666666666667, 17), (1839714, 'Q10914117', 0, 1, 66.66666666666667, 17), (1218725, 'Q707456', 0, 1, 66.66666666666667, 16), (1678795, 'Q6978828', 0, 1, 66.66666666666667, 16), (3113188, 'Q7676312', 0, 1, 66.66666666666667, 15), (3011958

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q1867', 'Q712937', 'Q244898', 'Q10914394', 'Q11063980']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q10914394', 'forw', 'direct'), ('Q712937', 'forw', 'direct'), ('Q244898', 'forw', 'direct'), ('Q1867', 'forw', 'direct'), ('Q11063980', 'forw', 'direct')]
ex_rels :
['P31', 'P961', 'P830', 'P47', 'P3591', 'P501', 'P1843', 'P960', 'P349', 'P910', 'P982', 'P2959', 'P3417', 'P646', 'P2184', 'P1376', 'P214', 'P18', 'P105', 'P1465', 'P1997', 'P268', 'P269', 'P281', 'P194', 'P2046', 'P150', 'P373', 'P227', 'P131', 'P473', 'P856', 'P948', 'P1464', 'P949', 'P41', 'P846', 'P1705', 'P225', 'P1792', 'P171', 'P3088', 'P6', 'P935', 'P2044', 'P17', 'P244', 'P94', 'P625', 'P1082', 'P1566', 'P158', 'P36', 'P1070', 'P1740', 'P421', 'P998', 'P300', 'P190', 'P242', 'P901']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Synonyms] not exist in w2v model: 桃園市
[Synonyms] not exist in w2v model: 宜蘭縣


,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P2046,[+271799700],0.680612,0.998231,0.681818,0.998231,1.680049,1
1,P625,"[Point(121.445833 25.011111), Point(121.633333...",0.563767,0.968611,0.582037,0.968611,1.550648,0
3,P421,[[UTC+8]],0.354152,0.984780,0.359626,0.984780,1.344406,0
4,P150,"[[林口區, 坪頂, 樹林口], [深坑區, 深坑仔, 簪纓], [中山區, 中山区], [...",0.340875,0.424962,0.802131,0.802131,1.227093,0
6,P2044,[+10],0.273419,0.606998,0.450445,0.606998,1.057443,0
7,P17,"[[中華民國, ROC, 中华台北, 中华民国, 中華台北, 台湾, 台灣, 民国, 民國]]",0.106260,0.340010,0.312520,0.340010,0.652530,0
8,P131,"[[中華民國, ROC, 中华台北, 中华民国, 中華台北, 台湾, 台灣, 民国, 民國]]",0.089268,0.285639,0.312520,0.312520,0.598159,0
9,P47,"[[新北市, NTPC, 北縣, 台北县, 台北縣, 台灣新北, 新北, 新台北, 臺北縣]...",0.058950,0.128337,0.459333,0.459333,0.587670,0
50,P1082,"[+2655515, +4054467]",0.075178,0.215409,0.349000,0.349000,0.564409,0
52,P105,"[[种, 物種, 種]]",0.030928,0.132738,0.233000,0.233000,0.365738,0


In [8]:
question = "日本有多少个县？"
a, b, c = do_search(question)
b

order_dict_for_merge :
{}
srtd_with_ratios :
[(1747461, 'Q17', 0, 1, 100.0, 7), (3378677, 'Q1640637', 0, 1, 100.0, 4), (1411706, 'Q2568549', 0, 1, 100.0, 4), (3763905, 'Q2335945', 0, 1, 100.0, 4), (3093782, 'Q1986642', 0, 1, 100.0, 4), (3048937, 'Q404510', 0, 1, 100.0, 4), (4263612, 'Q1683280', 0, 1, 100.0, 4), (2417121, 'Q3114534', 0, 1, 100.0, 3), (1641735, 'Q2732838', 0, 1, 100.0, 3), (4819510, 'Q543144', 0, 1, 100.0, 3), (6129095, 'Q1324085', 0, 1, 100.0, 3), (2169892, 'Q15097854', 0, 1, 100.0, 3), (5322024, 'Q2567839', 0, 1, 100.0, 3), (1078450, 'Q632162', 0, 1, 100.0, 3), (18651, 'Q845782', 0, 1, 100.0, 3), (1559171, 'Q1616990', 0, 1, 100.0, 3), (3519298, 'Q1082797', 0, 1, 100.0, 3), (1589231, 'Q570877', 0, 1, 100.0, 3), (5091938, 'Q3114533', 0, 1, 100.0, 3), (1105450, 'Q20725949', 0, 1, 100.0, 3), (1245004, 'Q3114532', 0, 1, 100.0, 3), (1010973, 'Q524663', 0, 1, 100.0, 3), (4217939, 'Q16639941', 0, 1, 100.0, 3), (5354788, 'Q1116749', 0, 1, 100.0, 3), (3651308, 'Q609160', 0, 1, 1

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q17', 'Q1640637', 'Q2568549', 'Q2335945', 'Q1986642']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q2568549', 'forw', 'direct'), ('Q1986642', 'forw', 'direct'), ('Q17', 'forw', 'direct'), ('Q1640637', 'forw', 'direct'), ('Q2335945', 'forw', 'direct')]
ex_rels :
['P3764', 'P349', 'P1151', 'P910', 'P982', 'P2959', 'P1814', 'P3417', 'P2250', 'P3762', 'P1792', 'P2299', 'P2853', 'P6', 'P237', 'P898', 'P94', 'P1906', 'P36', 'P163', 'P463', 'P1333', 'P2633', 'P3422', 'P242', 'P2134', 'P906', 'P2082', 'P402', 'P3183', 'P1313', 'P1245', 'P214', 'P1842', 'P373', 'P984', 'P2852', 'P2258', 'P527', 'P1332', 'P244', 'P832', 'P474', 'P3221', 'P1740', 'P421', 'P998', 'P1617', 'P641', 'P37', 'P31', 'P1296', 'P1343', 'P47', 'P1081', 'P30', 'P2184', 'P156', 'P1465', 'P269', 'P194', 'P2046', 'P2997', 'P227', 'P949', 'P2131', 'P298', 'P78', 'P3270', 'P487', 'P3106', 'P1198', 'P935', 'P17', '

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

rank_df :
                                                 cate      prob
0                                             日本下辖岩手县  0.499500
1                                             日本下辖佐贺县  0.499500
2                                             日本下辖山口县  0.499500
3                                             日本下辖熊本县  0.499500
4                                             日本下辖长崎县  0.499500
..                                                ...       ...
89  日本下辖http://www.wikidata.org/entity/statement/q...  0.271005
90  日本下辖http://www.wikidata.org/entity/statement/q...  0.271005
91  日本下辖http://www.wikidata.org/entity/statement/q...  0.271005
92  日本下辖http://www.wikidata.org/entity/statement/q...  0.271005
93  日本下辖http://www.wikidata.org/entity/statement/q...  0.271005

[94 rows x 2 columns]
rank_df_ori :
        prob                                               cate
0   0.958573                                            日本下辖兵库县
1   0.949639                                           日本

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P150,[47],0.515882,0.985773,0.523327,0.985773,1.5091,0


In [9]:
question = "埃尔达尔·梁赞诺夫出生在薩馬拉的时候他出生在哪个国家？"
a, b, c = do_search(question, {
        "埃尔达尔·梁赞诺夫": "Q381944"
    })
b

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'no_type')
entity_ids :
[['Q381944', 'Q597991', 'Q988626', 'Q504321', 'Q315255'], ['Q7485615', 'Q950834', 'Q4406294', 'Q894', 'Q80641']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q988626', 'forw', 'no_type'), ('Q504321', 'forw', 'no_type'), ('Q597991', 'forw', 'no_type'), ('Q381944', 'forw', 'no_type'), ('Q7485615', 'forw', 'no_type'), ('Q894', 'forw', 'no_type'), ('Q80641', 'forw', 'no_type'), ('Q950834', 'forw', 'no_type'), ('Q315255', 'forw', 'no_type'), ('Q4406294', 'forw', 'no_type')]
ex_rels :
['P1077', 'P27', 'P269', 'P361', 'P3537', 'P2046', 'P434', 'P106', 'P244', 'P1619', 'P2949', 'P163', 'P1618', 'P2019', 'P2067', 'P2387', 'P156', 'P373', 'P948', 'P454', 'P238', 'P3417', 'P1469', 'P571', 'P2561', 'P345', 'P102', 'P1448', 'P721', 'P17', 'P18', 'P764', 'P213', 'P2604', 'P1464', 'P735', 'P410', 'P41', 'P150', 'P6', 'P39', 'P1417', 'P166', 'P2020', 'P421', 'P2446', 'P242', 'P268', 'P227'

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


rank_df :
                                                cate      prob
0                          皮埃尔-夏尔·维尔纳夫所知最详细的出生地点瓦朗索尔  0.413846
1  大卫·梁赞诺夫所知最详细的出生地点http://www.wikidata.org/entit...  0.360016
2                               大卫·梁赞诺夫所知最详细的出生地点敖德薩  0.337213
3  埃尔-哈吉·迪乌夫所知最详细的出生地点http://www.wikidata.org/ent...  0.328435
4                             埃尔-哈吉·迪乌夫所知最详细的出生地点達喀爾  0.292857
5  皮埃尔-夏尔·维尔纳夫所知最详细的出生地点http://www.wikidata.org/e...  0.276150
rank_df_ori :
       prob                                               cate
0  0.502085                             埃尔-哈吉·迪乌夫所知最详细的出生地点達喀爾
1  0.340288  皮埃尔-夏尔·维尔纳夫所知最详细的出生地点http://www.wikidata.org/e...
2  0.337053  埃尔-哈吉·迪乌夫所知最详细的出生地点http://www.wikidata.org/ent...
3  0.336515                          皮埃尔-夏尔·维尔纳夫所知最详细的出生地点瓦朗索尔
4  0.301821                               大卫·梁赞诺夫所知最详细的出生地点敖德薩
5  0.104450  大卫·梁赞诺夫所知最详细的出生地点http://www.wikidata.org/entit...
merge :
                                                cate    prob_x    prob_y
0            

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P19,"[[苏联, 蘇聯], [俄罗斯苏维埃联邦社会主义共和国]]",0.330294,0.979946,0.337053,0.979946,1.316999,0


In [10]:
question = "费尔南多·佩索阿在什么情况下被称为Jean Seul de Méluret？"
a, b, c = do_search(question, {
        "Jean Seul de Méluret": "Q21028930"
    })
b

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'no_type')
entity_ids :
[['Q173481', 'Q311119', 'Q10514', 'Q295007', 'Q8915619'], ['Q21028930']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q173481', 'forw', 'no_type'), ('Q21028930', 'forw', 'no_type'), ('Q311119', 'forw', 'no_type'), ('Q10514', 'forw', 'no_type'), ('Q8915619', 'forw', 'no_type'), ('Q295007', 'forw', 'no_type')]
ex_rels :
['P27', 'P269', 'P434', 'P106', 'P244', 'P460', 'P1017', 'P1950', 'P950', 'P2949', 'P1315', 'P109', 'P1309', 'P648', 'P2067', 'P1284', 'P1953', 'P373', 'P1263', 'P2924', 'P3569', 'P691', 'P3417', 'P345', 'P102', 'P18', 'P213', 'P3373', 'P1477', 'P735', 'P2799', 'P1417', 'P39', 'P166', 'P22', 'P1050', 'P140', 'P1711', 'P25', 'P268', 'P227', 'P40', 'P1472', 'P1285', 'P1005', 'P1006', 'P1273', 'P910', 'P866', 'P1207', 'P1938', 'P2191', 'P535', 'P569', 'P2002', 'P2639', 'P509', 'P1899', 'P906', 'P108', 'P2163', 'P3430', 'P800', 'P1412', 'P2048', 'P949', 'P935', 'P

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

rank_df :
                                                cate      prob
0  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.361282
1  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.361282
2  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.361282
3  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.361282
4  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.361282
5  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.361282
6  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.353763
7  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.353763
8  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...  0.353763
rank_df_ori :
       prob                                               cate
0  0.042690  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...
1  0.036760  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...
2  0.036732  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...
3  0.030965  费尔南多·佩索阿别称http://www.wikidata.org/entity/state...
4  0.027290  费尔南多·佩索阿别称http://w

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P460,[[異名]],0.209702,0.580438,0.361282,0.580438,0.94172,0


In [11]:
question = "哪个城市下辖海曙区？"
a, b, c = do_search(question)
b    

order_dict_for_merge :
{'Q515': 0, 'Q24764': -1, 'Q2089229': -2}
srtd_with_ratios :
[(3420464, 'Q515', 0, 1, 100.0, 3), (4850169, 'Q24764', -1, 1, 100.0, 5), (969748, 'Q2089229', -2, 1, 100.0, 5), (4981246, 'Q317936', -3, 1, 100.0, 10), (3858130, 'Q11750', -3, 1, 100.0, 7), (853429, 'Q839602', -3, 1, 100.0, 6), (5412579, 'Q762726', -3, 1, 100.0, 6), (754208, 'Q757323', -3, 1, 100.0, 5), (1793017, 'Q478424', -3, 1, 100.0, 5), (5021163, 'Q729140', -3, 1, 100.0, 5), (5113858, 'Q1428625', -3, 1, 100.0, 5), (3144440, 'Q839066', -3, 1, 100.0, 5), (2090449, 'Q490055', -3, 1, 100.0, 5), (4596859, 'Q852579', -3, 1, 100.0, 5), (550955, 'Q133037', -3, 1, 100.0, 4), (5665084, 'Q919529', -3, 1, 100.0, 4), (1020228, 'Q109079', -3, 1, 80.0, 15), (4460778, 'Q1907114', -3, 1, 80.0, 10), (861845, 'Q1359958', -3, 1, 80.0, 7), (4077984, 'Q860251', -3, 1, 80.0, 7), (465280, 'Q1000142', -3, 1, 80.0, 7), (4153486, 'Q1375351', -3, 1, 80.0, 6), (2089776, 'Q274384', -3, 1, 80.0, 6), (6090681, 'Q10929912', -3, 1

order_dict_for_merge :
{}
srtd_with_ratios :
[(5887405, 'Q1026451', 0, 1, 100.0, 4), (3741275, 'Q27830394', 0, 1, 37.5, 4), (3837957, 'Q15904477', 0, 1, 37.5, 3), (4894726, 'Q11121663', 0, 1, 35.29411764705882, 4), (4094108, 'Q5296007', 0, 1, 35.29411764705882, 4), (4760110, 'Q14017704', 0, 1, 33.333333333333336, 4), (4217119, 'Q10766608', 0, 1, 31.57894736842105, 4), (5850277, 'Q10766617', 0, 1, 28.57142857142857, 4), (3276858, 'Q25922260', 0, 1, 24.0, 4)]
----------------------------------------------------------------------------------------------------
ori_entity_ids :
[['Q1026451', 'Q27830394', 'Q15904477', 'Q11121663', 'Q5296007']]
overload_entityid_list :
[]
ori_entity_ids :
[['Q1026451', 'Q27830394', 'Q15904477', 'Q11121663', 'Q5296007']]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q1026451', 'Q27830394', 'Q15904477', 'Q11121663', 'Q5296007']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q1026451', 'forw', 'direct'), ('Q11121663', 'forw', 'direct'), ('Q5296007', 'forw', 'direct'), ('Q15904477', 'forw', 'direct'), ('Q27830394', 'forw', 'direct')]
ex_rels :
['P31', 'P1566', 'P150', 'P373', 'P131', 'P17', 'P442', 'P421', 'P646', 'P1448', 'P625', 'P18']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

[Synonyms] not exist in w2v model: 
[Synonyms] not exist in w2v model: 章水镇
[Synonyms] not exist in w2v model: 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
1,P131,"[[宁波市, 宁波, 甬], [海曙区, 海曙]]",0.737819,0.954234,0.773205,0.954234,1.727439,1
0,P150,"[[高桥镇], [鄞江镇], [章水镇], [古林镇, 古林], [横街镇], [江厦街道,...",0.812819,0.970727,0.837330,0.970727,1.808057,0
20,P17,"[[中华人民共和国, 中国, 中國, 中華人民共和國, 共产中国, 新中国]]",0.289181,0.602150,0.480248,0.602150,1.082398,0
21,P625,"[Point(121.39475 29.85648), Point(121.41092 29...",0.254882,0.573021,0.444803,0.573021,1.017825,0
24,P421,[[UTC+8]],0.124112,0.407883,0.304282,0.407883,0.712165,0
25,P18,[http://commons.wikimedia.org/wiki/Special:Fil...,0.060284,0.121324,0.496883,0.496883,0.618207,0


In [12]:
question = "黄山市在哪个省？"
a, b, c = do_search(question)
b

order_dict_for_merge :
{}
srtd_with_ratios :
[(4638048, 'Q180470', 0, 1, 100.0, 5), (5711805, 'Q11177406', 0, 1, 66.66666666666667, 4), (4513405, 'Q11080976', 0, 1, 50.0, 3), (2081283, 'Q11081559', 0, 1, 50.0, 3), (3482712, 'Q11101199', 0, 1, 50.0, 3), (4593624, 'Q11178857', 0, 1, 50.0, 3), (1332509, 'Q11181826', 0, 1, 50.0, 3), (5961632, 'Q11130881', 0, 1, 50.0, 3), (4136512, 'Q10865478', 0, 1, 50.0, 3), (2082164, 'Q10878163', 0, 1, 50.0, 3), (5528958, 'Q10884327', 0, 1, 50.0, 3), (542895, 'Q10954472', 0, 1, 50.0, 3), (668416, 'Q11081998', 0, 1, 46.15384615384615, 3), (4902521, 'Q8526874', 0, 1, 40.0, 4), (2661832, 'Q10941385', 0, 1, 37.5, 6), (1491758, 'Q15959458', 0, 1, 35.29411764705882, 4), (5284833, 'Q15951760', 0, 1, 35.29411764705882, 4), (3928080, 'Q25924497', 0, 1, 31.57894736842105, 4), (2657117, 'Q21056374', 0, 1, 28.57142857142857, 4), (4170897, 'Q25922392', 0, 1, 25.0, 4), (1219070, 'Q25922396', 0, 1, 25.0, 4), (4793656, 'Q25922397', 0, 1, 24.0, 4), (4261176, 'Q25922434',

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q180470', 'Q11177406', 'Q11080976', 'Q11081559', 'Q11101199']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q11081559', 'forw', 'direct'), ('Q11177406', 'forw', 'direct'), ('Q180470', 'forw', 'direct'), ('Q11101199', 'forw', 'direct'), ('Q11080976', 'forw', 'direct')]
ex_rels :
['P31', 'P910', 'P646', 'P214', 'P81', 'P18', 'P281', 'P150', 'P373', 'P227', 'P131', 'P473', 'P856', 'P948', 'P442', 'P17', 'P625', 'P1566', 'P1378', 'P197', 'P421', 'P1448', 'P190']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P131,"[[安徽省, 安徽, 皖], [黄山区, 黃山區], [徽州区]]",0.809977,0.994932,0.814103,0.994932,1.809034,1
2,P150,"[[黟县], [歙县, 歙, 歙縣], [休宁县, 休宁], [祁门县, 祁門縣, 祁门],...",0.662397,0.926681,0.714805,0.926681,1.641486,0
4,P17,"[[中华人民共和国, 中国, 中國, 中華人民共和國, 共产中国, 新中国]]",0.605590,0.914665,0.662089,0.914665,1.576754,0
6,P625,"[Point(118.322 29.7229), Point(118.07878 30.50...",0.519553,0.865620,0.600210,0.865620,1.465830,0
17,P421,[[UTC+8]],0.212725,0.718826,0.295933,0.718826,1.014759,0
18,P197,[[篁墩站]],0.174275,0.338201,0.515301,0.515301,0.853502,0
19,P190,"[[因特拉肯, 煙特勒根]]",0.141725,0.378776,0.374167,0.378776,0.752943,0
20,P18,[http://commons.wikimedia.org/wiki/Special:Fil...,0.060397,0.099237,0.608615,0.608615,0.707852,0
21,P81,"[[皖赣铁路, 京贛鐵路, 皖贛鐵路, 皖赣线]]",0.064418,0.180818,0.356261,0.356261,0.537078,0
23,P948,[http://commons.wikimedia.org/wiki/Special:Fil...,0.000999,0.003045,0.328006,0.328006,0.331051,0


In [13]:
question = "苹果公司的总裁是谁？"
a, b, c = do_search(question)
b

order_dict_for_merge :
{}
srtd_with_ratios :
[(5978728, 'Q312', 0, 1, 100.0, 8), (4881362, 'Q1048090', 0, 1, 80.0, 3), (2923270, 'Q3150741', 0, 1, 80.0, 3), (2308663, 'Q3592722', 0, 1, 66.66666666666667, 3), (5509020, 'Q2984643', 0, 1, 61.53846153846154, 4), (377064, 'Q6630012', 0, 1, 57.14285714285714, 5), (4485018, 'Q18010946', 0, 1, 53.333333333333336, 5), (4725594, 'Q420932', 0, 1, 53.333333333333336, 5), (4134784, 'Q7852935', 0, 1, 53.333333333333336, 3), (785602, 'Q19837', 0, 1, 47.05882352941176, 17), (5871498, 'Q7215242', 0, 1, 47.05882352941176, 4), (5029824, 'Q6046437', 0, 1, 42.10526315789473, 4), (5389343, 'Q17843021', 0, 1, 38.095238095238095, 4), (5902766, 'Q13426339', 0, 1, 36.36363636363637, 4), (5868705, 'Q16973176', 0, 1, 34.78260869565217, 4), (2477496, 'Q25927330', 0, 1, 30.76923076923077, 3), (3219980, 'Q17842110', 0, 1, 27.586206896551722, 4)]
----------------------------------------------------------------------------------------------------
order_dict_for_merge 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

('forw', 'wiki', 'direct')
entity_ids :
[['Q312', 'Q1048090', 'Q3150741', 'Q3592722', 'Q2984643']]
----------------------------------------------------------------------------------------------------
queries_list: [('Q1048090', 'forw', 'direct'), ('Q312', 'forw', 'direct'), ('Q2984643', 'forw', 'direct'), ('Q3150741', 'forw', 'direct'), ('Q3592722', 'forw', 'direct')]
ex_rels :
['P31', 'P2740', 'P2037', 'P906', 'P1343', 'P2137', 'P213', 'P3222', 'P127', 'P349', 'P149', 'P1151', 'P910', 'P1167', 'P1297', 'P2959', 'P740', 'P3417', 'P946', 'P646', 'P1830', 'P214', 'P18', 'P268', 'P571', 'P3320', 'P245', 'P112', 'P159', 'P2174', 'P269', 'P3743', 'P1005', 'P373', 'P227', 'P949', 'P131', 'P856', 'P169', 'P2295', 'P1711', 'P3362', 'P2002', 'P1101', 'P154', 'P1454', 'P199', 'P1128', 'P2403', 'P409', 'P2427', 'P452', 'P414', 'P1278', 'P3500', 'P691', 'P935', 'P969', 'P1056', 'P17', 'P244', 'P488', 'P648', 'P2139', 'P625', 'P2657', 'P2226', 'P2138', 'P355', 'P3234', 'P921', 'P3761', 'P84', 'P239

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Synonyms] not exist in w2v model: 軟體
[Synonyms] not exist in w2v model: 業


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,pid,entity,multi_score,score,rank_score,max_score,sum_score,pid_in_cnt
0,P169,"[[提姆·庫克, 蒂姆·库克]]",0.633308,0.997930,0.634622,0.997930,1.632552,0
1,P3320,"[[史蒂夫·乔布斯, Steve Jobs, 乔布斯, 史提夫·喬布斯, 史提夫·賈伯斯, ...",0.563154,0.660649,0.852426,0.852426,1.513074,0
4,P84,"[[诺曼·福斯特, 諾曼·福士特, 霍朗明]]",0.345576,0.657055,0.525948,0.657055,1.183003,0
5,P112,"[[史蒂夫·乔布斯, Steve Jobs, 乔布斯, 史提夫·喬布斯, 史提夫·賈伯斯, ...",0.166090,0.304029,0.546297,0.546297,0.850325,0
15,P199,"[[蘋果商店, Apple Store, 苹果店, 苹果直销店, 苹果零售店, 蘋果專門店]]",0.129326,0.292593,0.442000,0.442000,0.734593,0
16,P452,"[[数字发行, 數位發行, 電子發行], [消費電子產品, 消費性電子, 消費性電子產品, ...",0.042357,0.107671,0.393391,0.393391,0.501062,0
19,P414,"[[纳斯达克, Nasdaq, 全国证券业协会行情自动传报系统, 納斯達克, 納斯達克股票交...",0.027522,0.067637,0.406914,0.406914,0.474551,0
22,P1830,[[Imagination Technologies]],0.046029,0.196123,0.234696,0.234696,0.430818,0
23,P1128,[+116000],0.038267,0.157576,0.242846,0.242846,0.400421,0
